## *<font color = maroon> Notebook 2: </font>* Detection of Sea Surface Temperature (SST) Filaments and Fronts - 
**This script includes the following:** <br> 
1. **Automated Detection**<br>
    - **Filaments** - *A parabolic dip in SST, where SST transitions sharply like a front, but with two sides and cool water in the middle*
        - Continous track - no gaps in data larger than 50 meters - has local minima in SST 
        - A SST gradient exceeding user set <font color = magenta> threshold </font>
        - Boat heading does not change more than 90$^{\circ}$ in structure
        - Symmetry in structure, i.e. change in SST on cooler side of asymmetrical filament shape is at least 30% of SST change for warmer side.
    - **Fronts** - *A sharp, step-like change in SST*
        - **If at least one of the filament defintions is not met, except for <font color = magenta> gradient threshold </font>, a filament is detected allowing for mutually exclusive recording of structures** 

2. **Data Ensemble** <br>
    - Individual structures, fronts and filaments, are stored in respective dictionaries for either all cruises or a <font color = magenta> user specified date </font>
    - Filaments
        - Normalized origin for all structures at the local SST minima
        - Limits of structure extend to bordering local maximas - hence parabolic shape
    - Fronts 
        - Normalized origin at peak SST gradient
        - Limits of fronts adjustable <font color = magenta> by length  </font> or default to neighboring minima and maxima.
    **For ensembles of all data, users may <font color = magenta> normalize shape </font> by keeping 'cool' sides (side of lesser change) to left for fronts (asymmetrical filaments)** <p>

3. **Saving Data - RECOMMENDED** <br>
    - It is recommended that ensembles of fronts and filaments of all cruises are <font color = magenta> saved in .pkl files </font> - `front_ensemble` & `fil_ensemble`, respectively - for use in future scripts. **This is necessary for plotting probabilities of structures across bay.**
    
<font color = maroon> **Code is organized based on the three sections defined above, with <font color = magenta> parameter </font> definitions below** </font>

In [ ]:
# --------- Edit Code --------- #

# Detection Parameters
window_size     = 25  # Multiply by 10 for physical distance (m) of smoothing relative to track
gradient_thresh = 1.0 # in deg. C per KM
extent          = .5 # Extent on either side a front's maximum gradient that is plotted. 

# Treating Data - true means removal (rmv)
rmv_nan = False 
rmv_mdr = True

# General Graphing Fontsizes
fs_tick  = 35
fs_label = 30 
fs_title = 40

# Individual Cruise or ALL Cruise Ensembles 
cruisedate = []   # Leave empty if no cruise 

# Normalize Cruise Data - set to false in code for indiviudal plots
normalize_ensembles = False

# Saving options
saveplots      = False
save_ensembles = True

#### Library Import

In [ ]:
# ----- Import Libraries ----- #

# Functions
import numpy as np
import functions as zf

# Files
import pickle

# Plotting
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
from matplotlib.collections import LineCollection
from matplotlib import cm
from matplotlib import colors
import matplotlib as mlb
from matplotlib import colors
from matplotlib.colors import LogNorm
from matplotlib.cm import get_cmap
from matplotlib.colors import Normalize, LogNorm, NoNorm
import matplotlib.patches as mpatches
import matplotlib.axes as AX
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
#from matplotlib.axes import inset_axes

# Mapping
import cartopy
import cartopy.crs as ccrs #importing the cartopy coordinate reference system library
import cartopy.feature as cfeature #importing the cartopy library of surface features
import cartopy.io.shapereader as shpreader

# Smoothing
import scipy.io as sio
import scipy.stats as ss
from scipy import signal
from scipy.signal import find_peaks
from scipy.signal import peak_prominences

# ----- IMPORT Track dictionary as pkl ----- #
# Read dictionary pkl file
if len(cruisedate) != 0:
    try:
        with open(f'cruise_{cruisedate[0]}.pkl', 'rb') as ztd:
            zodiac_track_data = pickle.load(ztd)
    except:
        with open('track_data.pkl', 'rb') as ztd:
            zodiac_track_data = pickle.load(ztd)
else:
    with open('track_data.pkl', 'rb') as ztd:
        zodiac_track_data = pickle.load(ztd)

In [ ]:
def is_subset(arr1, arr2):

    # Iterate over each element in the second array
    for i in range(len(arr2)):
        found = False

        # Check if the element exists in the first array
        for j in range(len(arr1)):
            if arr2[i] == arr1[j]:
                found = True
                break

        # If any element is not found, return false
        if not found:
            return False

    # If all elements are found, return true
    return True

### 1. Automated Detection 

#### Filaments 

In [ ]:
# Arrays to fill #
failed_dates   = [] # List of dates Zodiac did not leave
zero_grad      = [] # indices of where a sst local max and min resides 

# Arrays for Data Analysis # 
filament_width = []
fil_change     = []
symmetry       = []

# Ensemble of ALL filaments #
filament_ensemble = {}

# This master loop goes through each possible date the zodiac went out and grabs corresponding data 
num = 0 # For counting structures
for index, date in enumerate(zodiac_track_data):
        
    # If we only want one cruise, we reset ensemble each iteration until proper date is found and loop is exited
    # As we cycle through dates, we skip the rest of the loop until individual date is found
    if len(cruisedate) != 0:
        if cruisedate[0] == date:
            filament_ensemble = {}
            normalize_ensembles = False # Overwrites for individual plots 
        else:
            continue 

    # Reset subset array to ensure there are no copies of the same structure
    # NOTE: This means the same excat structure due to errors in surface sampler data, the same structure but at different
    # positions may be marked multiple times - no other way to know...
    subset_lat, subset_lon = [],[]

    try:
        lat, lon, SST, flu = zf.select_track(date, zodiac_track_data, excludenans=rmv_nan, excludemdr=rmv_mdr)
        # Data Processing, Filtering, & Gradients for SST #
        dist_processed, sst_processed = zf.data_processing(lat, lon, SST, 10)
        grad_dist_sst, grad_sst, lat_sst, lon_sst, sst_filtered, dist_filtered = \
        zf.running_avg_filter(window_size, lat, lon, sst_processed, dist_processed)
        # Data Processing, Filtering, & Gradients for Flu #
        dist_processed, flu_processed = zf.data_processing(lat, lon, flu, 10)
        grad_dist_flu, grad_flu, grad_lat_flu, grad_lon_flu, flu_filtered, dist_filtered = \
        zf.running_avg_filter(window_size, lat, lon, flu_processed, dist_processed)

        # Get final details on cruise path - Bearing (Direction of boat)
        bearing = np.around(zf.bearing(lat_sst, lon_sst), 2)
        # split this into components (vector form)
        lon_comp = 1*np.sin(np.around(bearing, 0)*(np.pi/180))
        lat_comp = 1*np.cos(np.around(bearing, 0)*(np.pi/180))

        # Find the local SST minimum and maximums #
        sst_loc_max, _ = find_peaks(sst_filtered, height=0) # READ ABOUT FIND PEAKS ####
        sst_loc_min, _ = find_peaks(-1*sst_filtered, height = -30)
        zero_grad = np.concatenate((sst_loc_max, sst_loc_min))
        # Brefily ensure they are ordered, as the minumums and maximum indices are appended out of order (not along track...)
        for i in range(0, len(zero_grad)):
            for j in range(i+1, len(zero_grad)):
                if(zero_grad[i] > zero_grad[j]):
                    temp = zero_grad[i];
                    zero_grad[i] = zero_grad[j];
                    zero_grad[j] = temp;

        # With our organized array of local maximum and minimum indices, we begin selecting individual segments
        start = 0 # Begin at 0th index
        for i in range(len(zero_grad)-2):    

            end = start+1                                     # Index to next one over 
            index1, index2 = zero_grad[start], zero_grad[end] # index 1 and 2 are now the indices that define 1/2 a segment 
            start = end                                       # Move start over 1 to grab next segment in loop

            # Ensure that we start with a dip
            if sst_filtered[index1] > sst_filtered[index2]:
                # If true, we become concerned in other portion, hence index 3, which is the peak following our central minima
                index3 = zero_grad[end+1]
                # Make sure that somewhere in our filament there is a sufficiently large gradient 
                if (np.max(grad_sst[index1:index3]) >= gradient_thresh):

                    # If true, check that we maintain a relatively straight bearing 
                    filament_xcomps, filament_ycomps = lon_comp[index1:index3], lat_comp[index1:index3]
                    X, Y = lon_comp[index1], lat_comp[index1]
                    angles = []
                    for i in range(len(filament_xcomps)-1):
                        try:
                            xi, yi = filament_xcomps[i+1], filament_ycomps[i+1]
                            angles = np.append(angles, (np.arccos(np.dot([X, Y], [xi, yi])/(np.sqrt(X**2 + Y**2)*np.sqrt(xi**2 + yi**2))))*(180/np.pi))
                        except:
                            angles = np.append(angles, np.mean(angles))
                        continue
                    # Ensuring that heading does not change more than 90 deg
                    if np.max(angles) < 90:    
                        # Save Filament Data # 
                        filament_distance = dist_filtered[index1:index3]
                        filament_sst      = sst_filtered[index1:index3]
                        filament_flu      = flu_filtered[index1:index3]
                        filament_lat      = lat_sst[index1:index3]
                        filament_lon      = lon_sst[index1:index3]

                        # order from 'small' to 'large' change in SST 
                        if normalize_ensembles == True:
                            if filament_sst[-1] < filament_sst[0]:
                                filament_sst = filament_sst[::-1]
                                filament_flu = filament_flu[::-1]
                            # This impacts how we measure symmetry 
                            symm_test = (filament_sst[0]-np.min(filament_sst))/(filament_sst[-1]-np.min(filament_sst))
                        else: 
                            # If we do not normalize, we still must ensure that symmetry test is done properly for 
                            # either case of asymmetry
                            if filament_sst[-1] < filament_sst[0]:
                                symm_test = (filament_sst[-1]-np.min(filament_sst))/(filament_sst[0]-np.min(filament_sst))
                            else:
                                symm_test = (filament_sst[0]-np.min(filament_sst))/(filament_sst[-1]-np.min(filament_sst))

                        # FINAL CHECKS - SYMMETRY AND DISCONTINUITIES
                        if (symm_test) >= .30:
                            if len(np.diff(filament_distance)[np.diff(filament_distance) > 0.05]) == 0:

                                # Normalize distances  
                                index          = np.where(filament_sst == np.min(filament_sst))
                                min_index      = index[0]
                                equal_dist     = filament_distance - filament_distance[min_index] # Equidistant View of Front from minima

                                # SST 
                                equal_sst = filament_sst - filament_sst[min_index]
                                fil_symm  = (filament_sst[0]-np.min(filament_sst))/(filament_sst[-1]-np.min(filament_sst))

                                # Positions of Filament (minima)
                                min_lat = filament_lat[min_index]
                                min_lon = filament_lon[min_index]
                                # Orientation of Filament
                                x_comp  = np.mean(filament_ycomps)
                                y_comp  = np.mean(-1*filament_xcomps)

                                # Ensure no repeat
                                if not is_subset(subset_lat, np.round(min_lat,4)) and not is_subset(subset_lon, np.round(min_lon,4)):
                                    # Set subset for this date to ensure same track is not checked twice
                                    subset_lat = np.append(subset_lat, np.round(min_lat, 4))
                                    subset_lon = np.append(subset_lon, np.round(min_lon,4))

                                    # Counter
                                    date_count = 1
                                    fil_number = num+1 # adjusts for pythons zero-based indexing
                                    num        = fil_number
                                    print(f'Filament {fil_number} Detected on {date}', end = '\r')
                                    
                                    # record stats
                                    filament_width = np.append(filament_width, abs(equal_dist[0])+abs(equal_dist[-1])) # Single value for size of front
                                    fil_change     = np.append(fil_change, np.abs(np.max(filament_sst)-np.min(filament_sst)))
                                    symmetry       = np.append(symmetry, fil_symm)                                    

                                    # Save details on filament in dictionary style
                                    cruises = {'Date': (date), \
                                               # Filament Imaging
                                               'Equi-Dist': (equal_dist), 'Equi-SST': (equal_sst), \
                                               # General Data
                                               'Distance': (filament_distance), 'SST': (filament_sst), 'Flu': (filament_flu), \
                                               'Latitude': (filament_lat), 'Longitude': (filament_lon), \
                                               'Min-Lat': (min_lat), 'Min-Lon': (min_lon), \
                                               'x-comp': (x_comp), 'y-comp': (y_comp), \
                                               'Symmetry': (fil_symm)}

                                    # Save to dictionary with corresponding number
                                    filament_ensemble[fil_number] = cruises

    except:
            # Dates may fail based on above logicals of nans - cruises where TSG or fluorometer did not record
            # and exclusion of MDR - cruises that did not leave marina
            failed_dates = np.append(failed_dates, date)
    continue 
                        

#### Fronts

In [ ]:
# Arrays to fill 
failed_dates = [] # List of dates Zodiac did not leave
zero_grad    = [] # indices of where a sst local max and min resides 

# Arrays for Data Analysis 
front_width   = []
front_change  = []

# Ensemble of ALL Fronts
front_ensemble = {}

# This master loop is simply to go through each possible date the zodiac went out, and grab corresponding data 
num = 0 # For counting fronts in our ensemble
for index, date in enumerate(zodiac_track_data):
        
    # If we only want one cruise, we reset ensemble each iteration until proper date is found and loop is exited
    # As we cycle through dates, we skip the rest of the loop until individual date is found
    if len(cruisedate) != 0:
        if cruisedate[0] == date:
            front_ensemble = {}
            normalize_ensembles = False # Overwrites for individual plots 
        else:
            continue 
            
    # Reset subset array to ensure there are no copies of the same structure
    # NOTE: This means the same excat structure due to errors in surface sampler data, the same structure but at different
    # positions may be marked multiple times - no other way to know...
    subset_lat, subset_lon = [],[]
        
    try: 
        lat, lon, SST, flu = zf.select_track(date, zodiac_track_data, excludenans=rmv_nan, excludemdr=rmv_mdr)

        # Data Processing, Filtering, & Gradients for SST
        dist_processed, sst_processed = zf.data_processing(lat, lon, SST, 10)
        grad_dist_sst, grad_sst, lat_sst, lon_sst, sst_filtered, dist_filtered = \
        zf.running_avg_filter(window_size, lat, lon, sst_processed, dist_processed)
        # Data Processing, Filtering, & Gradients for Flu
        dist_processed, flu_processed = zf.data_processing(lat, lon, flu, 10)
        grad_dist_flu, grad_flu, grad_lat_flu, grad_lon_flu, flu_filtered, dist_filtered = \
        zf.running_avg_filter(window_size, lat, lon, flu_processed, dist_processed)

        # Find the local SST minimum and maximums
        sst_loc_max, _ = find_peaks(sst_filtered, height=0)
        sst_loc_min, _ = find_peaks(-1*sst_filtered, height = -30)
        zero_grad = np.concatenate((sst_loc_max, sst_loc_min))
        # Brefily ensure they are ordered, as the minumums and maximums are out of order (not along track...)
        for i in range(0, len(zero_grad)):
            for j in range(i+1, len(zero_grad)):
                if(zero_grad[i] > zero_grad[j]):
                    temp = zero_grad[i];
                    zero_grad[i] = zero_grad[j];
                    zero_grad[j] = temp;

        # With our organized array of local maximum and minimums, we begin selecting individual segments
        start = 0 # Begin at 0th index
        for i in range(len(zero_grad)-1):
            end = start+1 # Index to next one over 
            index1, index2 = zero_grad[start], zero_grad[end] # index 1 and 2 are now the indices that define one segment 
            start = end # Move start over 1 to grab next segment in loop

            # With an individual segment selected, we grab its gradients and check whether it is a front
            nonzero_gradients = grad_sst[index1:index2]
            # since the front is of a large degree change, save its distance
            if np.max(nonzero_gradients) >= gradient_thresh:

                # Here, it has been proven that our segment exceeds a set threshold, and is indeed a front. Therefore grab other variables  
                nonzero_gradient_distances = grad_dist_sst[index1:index2] 
                frontal_gradient = nonzero_gradients
                frontal_distance = dist_filtered[index1:index2]           
                frontal_sst      = sst_filtered[index1:index2]
                frontal_flu      = flu_filtered[index1:index2]
                # Record the position too, using boat positions 
                frontal_lat = lat_sst[index1:index2]
                frontal_lon = lon_sst[index1:index2]

                # Ensure that this front is not a part of a filament
                filament = 0
                for f in range(len(filament_ensemble)):
                    if (filament_ensemble[f+1]['Date'] == date):
                        fil_lat = filament_ensemble[f+1]['Latitude']
                        fil_lon = filament_ensemble[f+1]['Longitude']
                        if is_subset(fil_lat, frontal_lat) and is_subset(fil_lon, frontal_lon):
                            #print(f'{dates[d]} number {(f)} is a filament')
                            filament = 1
                            break

                if filament != 1:
                    # We have a front and relevant information about that front. Before we proceed, lets make sure this front is 'legit'
                    # in some cases, data outages may creates large gaps in data, placing two different water readings next to each other 
                    if len(np.diff(frontal_distance)[np.diff(frontal_distance) > 0.05]) == 0:
                        # If there is not a large discontinuity across the bay, ensure that the removal of the marina has not created its own gap
                        mdr_front_lat = frontal_lat[((frontal_lon > -118.481) & (frontal_lon < -118.418)) & ((frontal_lat < 33.984) & (frontal_lat > 33.936))]
                        mdr_front_lon = frontal_lon[((frontal_lon > -118.481) & (frontal_lon < -118.418)) & ((frontal_lat < 33.984) & (frontal_lat > 33.936))]
                        # if there is data right outside of marina we exclude these fronts from the data, such that the loop does not conitnue
                        if (len(mdr_front_lat) == 0 ) and (len(mdr_front_lon) == 0):

                            # Organize from cold to warm water, if not already, such that all fronts are interpreted along one direction
                            if normalize_ensembles == True:
                                if frontal_sst[-1] < frontal_sst[0]:
                                    frontal_sst = frontal_sst[::-1]
                                    frontal_flu = frontal_flu[::-1]
                                    frontal_gradient = frontal_gradient[::-1]

                            # Equidistant View of Front, for common origin in ensemble
                            max_index  = np.where(frontal_gradient == np.max(frontal_gradient))[0]
                            
                            # Distances 
                            equal_dist = frontal_distance - frontal_distance[max_index-1]

                            # SST 
                            equal_sst  = frontal_sst - frontal_sst[max_index-1]

                            # Positions of front (max gradient)
                            max_lat = frontal_lat[max_index]
                            max_lon = frontal_lon[max_index]

                            # Ensure no repeat
                            if not is_subset(subset_lat, np.round(max_lat,5)) and not is_subset(subset_lon, np.round(max_lon,5)):
                                # Set subset for this date to ensure same track is not checked twice
                                subset_lat = np.append(subset_lat, np.round(max_lat, 5))
                                subset_lon = np.append(subset_lon, np.round(max_lon, 5))

                                # Counter
                                front_number = num+1
                                num = front_number
                                print(f'Front {front_number} Detected on {date}', end = '\r')
                                
                                # record stats 
                                front_width  = np.append(front_width, abs(equal_dist[0])+abs(equal_dist[-1])) # Single value for size of front
                                front_change = np.append(front_change, np.abs(equal_sst[-1]-equal_sst[0]))

                                # Save to Dictionary # 
                                          # Origin Based for Ensemble 
                                cruises = {'Date': (date), \
                                           # Frontal Imaging
                                           'Equi-Dist': (equal_dist), 'Equi-SST': (equal_sst), \
                                           # General Data
                                           'Distance': (frontal_distance), 'SST': (frontal_sst), 'Flu': (frontal_flu), \
                                           'Max-Lat': (max_lat), 'Max-Lon': (max_lon)}

                                front_ensemble[front_number] = cruises
        
    except:
        failed_dates = np.append(failed_dates, date)
    continue 

### 2. Plots (Ensembles)

#### Individual Plots

In [ ]:
if len(cruisedate) != 0:

    fig, ax = plt.subplots(1, figsize = (20,10))
    
    plt.plot(dist_filtered, sst_filtered)
    
    bearing = np.around(zf.bearing(lat_sst, lon_sst), 2)
    # split this into components (vector form)
    lon_comp = 1*np.sin(np.around(bearing, 0)*(np.pi/180))
    lat_comp = 1*np.cos(np.around(bearing, 0)*(np.pi/180))

    x = dist_filtered
    z = bearing 
    y = np.min(sst_filtered)*np.ones(len(dist_filtered)) - .01*np.min(sst_filtered)
    
    # Sort the x values and corresponding z values
    sorted_indices = np.argsort(x)
    x_sorted = x[sorted_indices]
    z_sorted = z[sorted_indices]
    
    # Create segments for the line
    points = np.array([x_sorted, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    # Normalize colors for the colormap
    norm = plt.Normalize(z_sorted.min(), z_sorted.max())
    cmap = plt.cm.twilight_shifted # Choose a colormap
    
    # Create a LineCollection
    line_collection = LineCollection(segments, cmap=cmap, norm=norm)
    line_collection.set_array(z_sorted[:-1])  # Set colors based on z values
    line_collection.set_linewidth(30)  # Set thickness of the line

    ax.add_collection(line_collection)
    
    if len(filament_ensemble) > 0:
        for i in range(len(filament_ensemble)):
            x=filament_ensemble[(i+1)]['Distance']
            y=filament_ensemble[(i+1)]['SST']    
            plt.plot(x, y, '-r')
    if len(front_ensemble) > 0:
        for i in range(len(front_ensemble)):
            x=front_ensemble[(i+1)]['Distance']
            y=front_ensemble[(i+1)]['SST']
            plt.plot(x, y, '-g')
    
    # Add a colorbar
    cbar = fig.colorbar(line_collection, ax=ax, orientation='vertical')
    cbar.set_label('Boat Heading - North = 0/360$^{\circ}$', fontsize = fs_tick)
    cbar.set_ticks(np.arange(0,420,60))
    cbar.ax.tick_params(labelsize=fs_tick-5)
    
    # Plot discontinuities 
    dist_track = []
    if len(np.diff(dist_filtered)[np.diff(dist_filtered) > 0.05]) != 0:
        breaks = np.where(np.diff(dist_filtered) > 0.05)[0]
        start = 0
        for b in range(len(breaks)):
            dist_track.append(dist_filtered[start:breaks[b]+1])
            start = breaks[b]+1
            if (b+1) == len(breaks):
                dist_track.append(dist_filtered[start::])
    for n in range(len(dist_track)): 
        if (n > 0) and (n < len(dist_track)):
            plt.axvspan(dist_track[n-1][-1], dist_track[n][0], color = 'gray', zorder = 0)

    green_patch = mpatches.Patch(color='green', label='Front') 
    red_patch = mpatches.Patch(color='red', label='Filament')
    gray_patch = mpatches.Patch(color='gray', label='Discontinuity')
    plt.legend(handles=[green_patch, red_patch, gray_patch], loc='upper left', fontsize = fs_tick)
    
    plt.xticks(fontsize=fs_tick)
    plt.yticks(fontsize=fs_tick)
        
    #plt.title(f'Automated Detection of SST Structures - {cruisedate[0]}', fontsize=fs_title)
    plt.title(f'One Dimensional Track for {cruisedate[0]}', fontsize=fs_title)
    plt.xlabel('Distance [km]', fontsize = fs_label)
    plt.ylabel('SST [$^{\circ}C$]', fontsize = fs_label)
    plt.ylabel('Fluorescence [counts]', fontsize = fs_label)
    
    plt.tight_layout()

if len(cruisedate) != 0:
    if saveplots == True:
        filename = f'sst_track_{cruisedate[0]}.png'
        plt.savefig(filename)

#### Fluorescence Plots

In [ ]:
# Mapped Individual Cruise #        
if len(cruisedate) != 0 and rmv_nan:
    
    # scale plots 
    if max(lat_sst) > 33.9:
        nrows = 2
        ncols = 1
        dims = (30,15)
        cbar_scale = .8
    if min(lat_sst) < 33.8:
        nrows = 1
        ncols = 2
        dims = (20,15)
        cbar_scale = .7
    
    track_fig, ax = plt.subplots(nrows,ncols,subplot_kw={'projection': ccrs.PlateCarree()}, figsize = dims)
        
    for plot in range(2):
    
        ax = plt.subplot(nrows, ncols, plot+1)

        long_lat_coords = [-118.85, -118.25, 33.7, 34.05]
        ax.set_extent(long_lat_coords)
        ax.coastlines()
        ax.add_feature(cfeature.OCEAN)
        ax.add_feature(cfeature.LAND)
        ax.add_feature(cfeature.COASTLINE)
        
        if rmv_mdr == True:
            vertices = [-118.48, -118.417, 33.983902, 33.935]
            x_origin, y_origin = vertices[0], vertices[2]
            x_distance, y_distance = (vertices[1] - vertices[0]), (vertices[3] - vertices[2])
            rectangle = mpatches.Rectangle((x_origin, y_origin), x_distance, y_distance, fc='grey',ec="red")
            plt.gca().add_patch(rectangle)
            plt.axis('scaled')

        if plot == 0:
            plt.scatter(lon_sst, lat_sst, c = sst_filtered, cmap='jet')
            plt.title('SST Track - $^{\circ}$C', fontsize=fs_label)
            cbar = plt.colorbar(shrink = cbar_scale)
            cbar.ax.tick_params(labelsize=fs_tick-10)
            for i in range(len(filament_ensemble)):
                #plt.scatter(filament_ensemble[i+1]['Min-Lon'], filament_ensemble[i+1]['Min-Lat'], s = 100, color = 'red')
                plt.text(filament_ensemble[i+1]['Min-Lon'], filament_ensemble[i+1]['Min-Lat'], f'{i+1}', \
                         fontsize = fs_label, color = 'magenta')
            for i in range(len(front_ensemble)):
                #plt.scatter(front_ensemble[i+1]['Max-Lon'], front_ensemble[i+1]['Max-Lat'], s = 100, color = 'magenta')
                plt.text(front_ensemble[i+1]['Max-Lon'], front_ensemble[i+1]['Max-Lat'], f'{i+1}', \
                         fontsize = fs_label, color = 'black')
                
        else:
            plt.scatter(lon_sst, lat_sst, c = flu_filtered, cmap = 'Greens')
            plt.title('Flourescence Track - Raw Counts', fontsize=fs_label)
            cbar = plt.colorbar(shrink = cbar_scale)
            cbar.set_ticks(np.linspace(np.round(min(flu_filtered),0),np.round(max(flu_filtered),0),5))
            cbar.ax.tick_params(labelsize=fs_tick-10)

        plt.xlim(min(lon_sst)-.01, max(lon_sst)+.01)
        plt.ylim(min(lat_sst)-.01, max(lat_sst)+.01)
                
    plt.tight_layout()
    
if len(cruisedate) != 0:
    if saveplots == True:
        filename = f'mapped_cruise_{cruisedate[0]}.png'
        plt.savefig(filename)

In [ ]:
# Individual Structure Plots - OPTIONAL 
if len(cruisedate) != 0 and rmv_nan:
    
    from scipy.signal import find_peaks, peak_prominences

    # from above mapped track, select between front or filament and map said specific feature by number 
    front  = True 
    number = 6
        
    fig, ax1 = plt.subplots(1, figsize = (30,13))
    ax2 = ax1.twinx()
    
    if front:
        dis = front_ensemble[number]['Equi-Dist']
        sst = front_ensemble[number]['SST']
        flu = front_ensemble[number]['Flu']
    else:
        dis = filament_ensemble[number]['Equi-Dist']
        sst = filament_ensemble[number]['SST']
        flu = filament_ensemble[number]['Flu']
        
    # norm flu
    flu = (flu - min(flu)) / (max(flu) -min(flu))
    
    # Plotting
    ax1.plot(dis, sst, color = 'lightcoral')
    ax2.plot(dis, flu, color = 'seagreen')
    max_flu_index = np.where(flu == np.max(flu))[0]
    ax1.axvline(dis[max_flu_index], color = 'gray')
    
    # Plot peaks
    peaks, _ = find_peaks(flu)
    prominences = peak_prominences(flu, peaks)[0]
    max_prom = peaks[np.where(prominences == np.max(prominences))[0]]
    ax1.axvline(dis[max_prom], color = 'gray')

    # Edit plot
    ax1.set_xlabel('Distance from Center', fontsize = fs_label)
    ax1.set_ylabel('SST', fontsize = fs_label)
    ax2.set_ylabel('Fluorescence [Normalized]', fontsize = fs_label)
    ax1.yaxis.label.set_color('lightcoral')
    ax2.yaxis.label.set_color('seagreen')
    ax1.tick_params(axis='both', labelsize=fs_tick)
    ax2.tick_params(axis='both', labelsize=fs_tick)
    
    # Label peak prominences if any
    for i in range(len(peaks)):
        ax2.plot(dis[peaks[i]], flu[peaks[i]], 'g*')
        ax2.text(dis[peaks[i]], flu[peaks[i]], f'{prominences[i]: .2f}', fontsize = 20)

In [ ]:
if rmv_nan and len(cruisedate) != 0:
    
    dis_prom_front, val_prom_front = [], []
    dis_prom_fil, val_prom_fil = [], []
        
    # Go through all filaments and fronts and calculate peak prominence position and its respective value

    # 1 - Select between fronts (0) and filaments (1)
    for structure in range(2):
        
        if structure == 0:
            for number in range(len(front_ensemble)):
                dis = front_ensemble[number+1]['Equi-Dist']
                sst = front_ensemble[number+1]['SST']
                flu = front_ensemble[number+1]['Flu']
                
                # 2 - Normalize fluorescence and distance 
                # 2.1: Flu
                flu = (flu - min(flu)) / (max(flu) -min(flu))
                # 2.2: dis
                #dis[dis < 0] = -1* ((dis[dis < 0]) / (min(dis[dis < 0])))
                #dis[dis > 0] = ((dis[dis > 0]) / (max(dis[dis > 0])))

                # 3 - Calc promincences and find highes ones distance from structure
                try:
                    peaks, _ = find_peaks(flu)
                    prominences = peak_prominences(flu, peaks)[0]
                    max_prom = peaks[np.where(prominences == np.max(prominences))[0]]
                    # record values 
                    #dis_prom = np.append(dis_prom, dis[max_prom])
                    #val_prom = np.append(val_prom, np.max(prominences))
                    if len(max_prom) != 2:
                        # record values 
                        dis_prom_front = np.append(dis_prom_front, dis[max_prom])
                        val_prom_front = np.append(val_prom_front, np.max(prominences))
                except:
                    #print(f'No peak for front {number+1}')
                    pass
        else:
            for number in range(len(filament_ensemble)):
                dis = filament_ensemble[number+1]['Equi-Dist']
                sst = filament_ensemble[number+1]['SST']
                flu = filament_ensemble[number+1]['Flu']
        
                # 2 - Normalize fluorescence and distance 
                # 2.1: Flu
                flu = (flu - min(flu)) / (max(flu) -min(flu))
                # 2.2: dis
                #dis[dis < 0] = -1* ((dis[dis < 0]) / (min(dis[dis < 0])))
                #dis[dis > 0] = ((dis[dis > 0]) / (max(dis[dis > 0])))

                # 3 - Calc promincences and find highes ones distance from structure
                try:
                    peaks, _ = find_peaks(flu)
                    prominences = peak_prominences(flu, peaks)[0]
                    max_prom = peaks[np.where(prominences == np.max(prominences))[0]]
                    # record values 
                    dis_prom_fil = np.append(dis_prom_fil, dis[max_prom])
                    val_prom_fil = np.append(val_prom_fil, np.max(prominences))
                    if len(dis_prom) != len(val_prom):
                        print(structure, number)
                        break
                except:
                    #print(f'No peak for front {number+1}')
                    pass

In [ ]:
if rmv_nan and len(cruisedate) != 0:
    
    fig, ax = plt.subplots(1, figsize = (25,13))

    # Plot data 
    plt.scatter(np.abs(dis_prom_front), val_prom_front, s = 100, color = 'k', label = 'Front')
    plt.scatter(np.abs(dis_prom_fil), val_prom_fil, s = 100, color = 'magenta', label = 'Filament')
    
    # Add bounds
    ax.axvline(0, color = 'gray', linestyle = '--')
    ax.axhline(0, color = 'gray', linestyle = '--')
    
    plt.legend(fontsize = fs_tick)
    
#     ax.axvline(-1, color = 'blue', linestyle = '--')
#     ax.text(-1, .8, 'Side of filament of < $\Delta_{SST}$', \
#              color = 'blue', rotation = 270, verticalalignment='top', horizontalalignment='left', fontsize = fs_tick)
#     ax.axvline(1, color = 'red', linestyle = '--')
#     ax.text(1, .8, 'Side of filament of > $\Delta_{SST}$', \
#              color = 'red', rotation = 270, verticalalignment='top', horizontalalignment='right', fontsize = fs_tick)
 
    # edit graph
    plt.xlim(-.01, 1.01)
    plt.ylim(-.05, 1.01)
    ax.tick_params(axis='both', labelsize=fs_tick)
    ax.set_xlabel('Distance from Center [kilometers]', fontsize = fs_label)
    ax.set_ylabel('Peak Prominence', fontsize = fs_label)
    plt.title(f'Peak Prominence versus Distance from Center', fontsize = fs_title)
    
    plt.tight_layout()

#### Ensembles

In [ ]:
fil_ens_fig = plt.figure(1, (20, 10)) 

# extent of ensemble in km from max gradient
extent = .5

plt.axhline(y=0, linestyle = '--', color = 'k', alpha = .5)
plt.axvline(x=0, linestyle = '--', color = 'k', alpha = .5)

for i in range(len(filament_ensemble)):
    x = filament_ensemble[i+1]['Equi-Dist']
    y = filament_ensemble[i+1]['Equi-SST']

    if max(y) <= .5:
        plt.plot(x, y, color = 'green', zorder=4)
    elif max(y) <= 1.0:
        plt.plot(x, y, color = 'blue', zorder=3)
    elif max(y) <= 1.5:
        plt.plot(x, y, color = 'orange', zorder=2)
    else:
        plt.plot(x, y, color = 'red', zorder=1)
        
plt.xlim(-1, 1)
plt.ylim(-.1, 1.7)

plt.xlabel('Distance from Minimum SST [km]', fontsize=fs_label)
plt.ylabel('$\Delta$SST from Minimum [$^{\circ}C$]', fontsize=fs_label)

if len(cruisedate) != 0:
    plt.title(f'{cruisedate[0]} Ensemble of Filaments - {len(filament_ensemble)} Total', fontsize=fs_title)
else:
    plt.title(f'Ensemble of Filaments - {len(filament_ensemble)} Total', fontsize=fs_title)

green_patch = mpatches.Patch(color='green', label='$\Delta_{max}$ $\leq$ .5') 
blue_patch = mpatches.Patch(color='blue', label='.5 < $\Delta_{max}$ $\leq$ 1.0')
orange_patch = mpatches.Patch(color='orange', label='1.0 < $\Delta_{max}$ $\leq$ 1.5')
red_patch = mpatches.Patch(color='red', label='$\Delta_{max}$ > 1.5')
plt.legend(handles=[green_patch, blue_patch, orange_patch, red_patch], fontsize = fs_tick-10, loc = 'upper left')

plt.xticks(fontsize=fs_tick), plt.yticks(fontsize=fs_tick)

plt.tight_layout()

#print(f'Cruise Dates: \n {dates}')

if saveplots == True:
    plt.savefig('all_filament_ensemble.png')

In [ ]:
front_ens_subplots, ax = plt.subplots(2, 2, figsize=(18, 12))  # 3 rows, 1 colum

dates = []
uhlf_km, ohlf_km, over_km, ovr_hkm = 0,0,0,0

sum = 0
for i in range(len(front_ensemble)):
    x = front_ensemble[i+1]['Equi-Dist']
    y = front_ensemble[i+1]['Equi-SST']
    # Focus on certain part of front
    y = y[(x >= -1*extent) & (x <= extent)]
    x = x[(x >= -1*extent) & (x <= extent)]
    if front_width[i] > 5:
        continue
    sum = i + 1
    if front_change[i] <= .5:
        ax[0,0].plot(x, y, color = 'green', zorder=4, alpha = .7)
        uhlf_km = uhlf_km+1
    elif front_change[i] <= 1:
        ax[0,1].plot(x, y, color = 'blue', zorder=3, alpha = .7)
        ohlf_km = ohlf_km+1
    elif front_change[i] <= 1.5:
        ax[1,0].plot(x, y, color = 'orange', zorder=2)
        over_km = over_km+1
    else:
        ax[1,1].plot(x, y, color = 'red', zorder=1)
        ovr_hkm = ovr_hkm+1
        
dates = np.unique(date)

title_strings = ['$\Delta_{sst} \leq .5^{\circ}C$', '$.5^{\circ}C$ < $\Delta_{sst}$ $\leq$ 1.0$^{\circ}C$', \
                 '$1.0^{\circ}C$ < $\Delta_{sst}$ $\leq$ $1.5^{\circ}C$', '$\Delta_{sst}$ > $1.5^{\circ}C$']
number_front_perplot = [uhlf_km, ohlf_km, over_km, ovr_hkm]

k = 0
for i in range(2):
    for j in range(2):
        ax[i,j].axhline(0, linestyle = '--', color = 'k', alpha = .5)
        ax[i,j].axvline(0, linestyle = '--', color = 'k', alpha = .5)
        ax[i,j].set_xlim(-1*extent,extent)
        ax[i,j].tick_params(axis='both', labelsize=15)
        ax[i,j].set_title(title_strings[k], fontsize = 20)
        ax[i,j].text(-.48, 0, f'{number_front_perplot[k]}', fontsize=fs_label, va = 'bottom')
        k = k + 1
        
front_ens_subplots.suptitle('Front Ensemble by Total Change in SST ($\Delta_{sst}$)' f' - {np.sum(number_front_perplot)} Total', fontsize = fs_label)
front_ens_subplots.supxlabel('Distance from Maximum SST Gradient [km]', fontsize = fs_label)   
front_ens_subplots.supylabel('$\Delta$SST from Max Gradient [$^{\circ}C$]', fontsize = fs_label)
    
    
plt.tight_layout()

if saveplots == True:
    plt.savefig('front_ens_subplots.png')

### 3. Saving Ensembles

In [ ]:
if save_ensembles:    
    import pickle
    with open('filament_ensemble.pkl', 'wb') as ztd:
        pickle.dump(filament_ensemble, ztd)
        print('filament ensemble saved successfully to file')
    with open('front_ensemble.pkl', 'wb') as ztd:
        pickle.dump(front_ensemble, ztd)
        print('front ensemble saved successfully to file')